<a href="https://colab.research.google.com/github/SelenaNahra/DL/blob/main/HW3P2LSTM30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LSTM - 30 Sequences**

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import requests
import time

# Step 1: Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Step 2: Prepare the dataset
sequence_length = 30
# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Step 3: Create a dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Step 4: Create data loaders
batch_size = 128
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Defining the RNN model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        return output

# Hyperparameters
hidden_size = 128
learning_rate = 0.005
epochs = 100

# Model, loss, and optimizer
model = CharRNN(len(chars), hidden_size, len(chars)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
start_time = time.time()

for epoch in range(epochs):
    model.train()
    for sequences, targets in train_loader:  # Iterate over batches
        sequences, targets = sequences.to(device), targets.to(device)  # Move data to device
        optimizer.zero_grad()
        output = model(sequences)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for sequences, targets in test_loader:
            sequences, targets = sequences.to(device), targets.to(device)  # Move data to device
            val_output = model(sequences)
            val_loss += criterion(val_output, targets).item()
            _, predicted = torch.max(val_output.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        val_loss /= len(test_loader)
        val_accuracy = correct / total

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

end_time = time.time()
training_time = end_time - start_time

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-sequence_length:]], dtype=torch.long).unsqueeze(0).to(device)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Counting model complexity
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")
print(f"Training time: {training_time} seconds")
print(f"Number of trainable parameters in the model: {count_parameters(model)}")

# Compute computational complexity
embedding_complexity = sequences.shape[1] * hidden_size
rnn_complexity = sequences.shape[1] * hidden_size * hidden_size
linear_complexity = hidden_size * len(chars)

total_complexity = embedding_complexity + rnn_complexity + linear_complexity
print(f"Total computational complexity: {total_complexity}")

Epoch 10, Loss: 3.5751774311065674, Validation Loss: 1.5580929978715916, Validation Accuracy: 0.5303600166761553
Epoch 20, Loss: 0.8811764717102051, Validation Loss: 1.597962789540994, Validation Accuracy: 0.5176870351857912
Epoch 30, Loss: 0.4532249867916107, Validation Loss: 1.6532915059821194, Validation Accuracy: 0.5074437515970108
Epoch 40, Loss: 1.3330286741256714, Validation Loss: 1.6830997605879143, Validation Accuracy: 0.49856773343255345
Epoch 50, Loss: 1.2158069610595703, Validation Loss: 1.6849483785038355, Validation Accuracy: 0.5007508752740134
Epoch 60, Loss: 0.2549244463443756, Validation Loss: 1.6835572686745255, Validation Accuracy: 0.49792668767623155
Epoch 70, Loss: 1.7348018884658813, Validation Loss: 1.7146450645678768, Validation Accuracy: 0.49286108135005136
Epoch 80, Loss: 1.8756173849105835, Validation Loss: 1.7209599756743124, Validation Accuracy: 0.48907308369905816
Epoch 90, Loss: 1.9917711019515991, Validation Loss: 1.7309278679249687, Validation Accuracy:

**LSTM Adjusted Hyperparameters - 30 Sequences**

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import requests
import time

# Step 1: Download the dataset
url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
response = requests.get(url)
text = response.text  # This is the entire text data

# Step 2: Prepare the dataset
sequence_length = 30
# Create a character mapping to integers
chars = sorted(list(set(text)))
char_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_char = {i: ch for i, ch in enumerate(chars)}

# Encode the text into integers
encoded_text = [char_to_int[ch] for ch in text]

# Create sequences and targets
sequences = []
targets = []
for i in range(0, len(encoded_text) - sequence_length):
    seq = encoded_text[i:i+sequence_length]
    target = encoded_text[i+sequence_length]
    sequences.append(seq)
    targets.append(target)

# Convert lists to PyTorch tensors
sequences = torch.tensor(sequences, dtype=torch.long)
targets = torch.tensor(targets, dtype=torch.long)

# Step 3: Create a dataset class
class CharDataset(Dataset):
    def __init__(self, sequences, targets):
        self.sequences = sequences
        self.targets = targets

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        return self.sequences[index], self.targets[index]

# Instantiate the dataset
dataset = CharDataset(sequences, targets)

# Step 4: Create data loaders
batch_size = 128
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

# Defining the RNN model
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, fc_hidden_size):
        super(CharRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, fc_hidden_size)
        self.fc2 = nn.Linear(fc_hidden_size, output_size)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        output = self.fc2(output)
        return output

# Hyperparameters with adjustable FCNN, hidden layers, and hidden states
hidden_size = 256  # Increase the hidden size
num_layers = 3  # Increase the number of hidden layers
fc_hidden_size = 512  # Increase the hidden size of the fully connected layer

learning_rate = 0.005
epochs = 100

# Model, loss, and optimizer
model = CharRNN(len(chars), hidden_size, len(chars), num_layers, fc_hidden_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
start_time = time.time()

for epoch in range(epochs):
    model.train()
    for sequences, targets in train_loader:  # Iterate over batches
        sequences, targets = sequences.to(device), targets.to(device)  # Move data to device
        optimizer.zero_grad()
        output = model(sequences)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_loss = 0
        correct = 0
        total = 0
        for sequences, targets in test_loader:
            sequences, targets = sequences.to(device), targets.to(device)  # Move data to device
            val_output = model(sequences)
            val_loss += criterion(val_output, targets).item()
            _, predicted = torch.max(val_output.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        val_loss /= len(test_loader)
        val_accuracy = correct / total

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}')

end_time = time.time()
training_time = end_time - start_time

# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-sequence_length:]], dtype=torch.long).unsqueeze(0).to(device)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction, dim=1).item()
        return ix_to_char[predicted_index]

# Predicting the next character
test_str = "This is a simple example to demonstrate how to predict the next char"
predicted_char = predict_next_char(model, char_to_int, int_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")
print(f"Training time: {training_time} seconds")
print(f"Number of trainable parameters in the model: {count_parameters(model)}")

# Compute computational complexity
embedding_complexity = sequences.shape[1] * hidden_size
rnn_complexity = sequences.shape[1] * hidden_size * hidden_size * num_layers
linear_complexity = fc_hidden_size * hidden_size + fc_hidden_size * len(chars)

total_complexity = embedding_complexity + rnn_complexity + linear_complexity
print(f"Total computational complexity: {total_complexity}")

Epoch 10, Loss: 0.6489378213882446, Validation Loss: 1.4088479500013866, Validation Accuracy: 0.5708714187732267
Epoch 20, Loss: 1.6172584295272827, Validation Loss: 1.3791037553422285, Validation Accuracy: 0.5817736794681562
Epoch 30, Loss: 1.793871283531189, Validation Loss: 1.3789208641492698, Validation Accuracy: 0.5834906062141093
Epoch 40, Loss: 1.8068017959594727, Validation Loss: 1.3851009797790033, Validation Accuracy: 0.5829168030196393
Epoch 50, Loss: 2.0704314708709717, Validation Loss: 1.3793348310599707, Validation Accuracy: 0.5852792583593711
Epoch 60, Loss: 0.07259982079267502, Validation Loss: 1.3895109883000367, Validation Accuracy: 0.5854182263205319
Epoch 70, Loss: 1.036695122718811, Validation Loss: 1.3987508555697903, Validation Accuracy: 0.5831364620550223
Epoch 80, Loss: 0.23537789285182953, Validation Loss: 1.4011450401340075, Validation Accuracy: 0.5834771577017389
Epoch 90, Loss: 0.1842183917760849, Validation Loss: 1.4069960900441847, Validation Accuracy: 0.